In [1]:
%matplotlib inline

In [2]:
import torch
import os
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import finalprojectneuralnetworks as fpnn
import csv
import time

In [3]:
n = 5  # number of samples/batch size
l = 5  # sentence len
d = 3  # embedding dimension
rand_arr = torch.rand(n, l, d)

In [4]:
layer = nn.Conv1d(1, 2, 3, stride=1, padding=0)
print("layer: ", layer)
input = torch.randn(1, 140)
print("Input: ", input.size())
input = input.unsqueeze(0)
print("Input unsqueezed: ", input.size())
print(input)
output = layer(input)
print("output.size: ", output.size())
print("output: ", output)

layer:  Conv1d(1, 2, kernel_size=(3,), stride=(1,))
Input:  torch.Size([1, 140])
Input unsqueezed:  torch.Size([1, 1, 140])
tensor([[[ 0.9818, -0.5621,  0.6250,  1.1725, -0.6883, -1.1592, -1.0358,
           1.5947, -0.3664, -0.5379,  0.1732,  0.1164, -0.1166,  0.9325,
           1.1040, -1.3489, -0.1892,  0.9851,  1.3359, -0.4393, -0.4778,
           0.3597,  1.4018, -0.9472, -0.0310, -0.0696,  1.0199,  1.1905,
           1.9292,  0.4771,  1.6502, -0.3107,  0.0039,  0.7897,  0.2355,
          -0.0045,  1.7045, -1.5602,  0.9935,  0.9516,  2.0607, -0.0906,
          -0.4886,  2.1094,  0.5660, -0.5469, -0.0251, -0.0609, -0.7429,
           0.8930,  0.1181,  1.3321,  0.3881,  0.4368, -0.9118,  1.7250,
           0.1298,  0.1148, -0.4777, -0.9992,  0.8362,  0.5460, -0.0478,
           0.5141,  0.0694,  0.1780, -1.1466, -0.4028, -1.9904,  1.3534,
           1.4870,  1.6848,  1.3210,  1.1253,  1.8726, -0.7751,  0.2243,
           0.5229,  0.2561,  0.4091, -2.8773, -0.0865, -0.3889, -0.5978,


In [5]:
root_directory = "D:\College\CS_545\Final_Project"
# root_directory = "/s/chopin/l/grad/acf003/CS_545"

train_file = "EEG_Eye_State_Lumped_10_Normalized_Train.csv"
validate_file = "EEG_Eye_State_Lumped_10_Normalized_Validate.csv"
test_file = "EEG_Eye_State_Lumped_10_Normalized_Test.csv"

train_dataset = fpnn.FinalProjectEEGDataset(train_file, root_directory)
validate_dataset = fpnn.FinalProjectEEGDataset(validate_file, root_directory)
test_dataset = fpnn.FinalProjectEEGDataset(test_file, root_directory)
train_dataloader = DataLoader(train_dataset, batch_size=64)
validate_dataloader = DataLoader(validate_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

num_inputs = 140
num_channels = 1
kernel_size = 3
num_outputs = 2
num_outputs2 = 5


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(num_channels, num_outputs, kernel_size)
        self.conv2 = nn.Conv1d(num_outputs, num_outputs2, kernel_size)
        print((kernel_size - 1) * 2)
        self.fc1 = nn.Linear(
            (num_inputs - (kernel_size - 1) * 2) * num_outputs2, num_outputs
        )
        self.fc2 = nn.Linear(num_outputs, num_outputs)

    def forward(self, x):
        # print("x.shape1", x.shape)
        x = nn.functional.relu(self.conv1(x))
        # print("x.shape2", x.shape)

        x = nn.functional.relu(self.conv2(x))
        # print("x.shape3", x.shape)

        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        # print("x.shape4", x.shape)

        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Net().float()

4


In [7]:
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Net(
  (conv1): Conv1d(1, 2, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(2, 5, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=680, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=2, bias=True)
)


In [8]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = loss_fn(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print("Finished Training")

Finished Training


In [9]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in validate_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy of the network on the test inputs: %d %%" % (100 * correct / total))

Accuracy of the network on the test inputs: 50 %
